In [1]:
!pip install accelerate peft datasets bitsandbytes evaluate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 22.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.2 MB/s eta 0:00:00


In [2]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=b44f75b663fb9ebf0e65e584ac55191497922d9b834594ed30e9399703136486
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [3]:
import evaluate
import torch
from torch.utils.data import DataLoader
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    default_data_collator,
)
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from tqdm import tqdm
import os

In [4]:
from huggingface_hub import login

hf_token = os.getenv("HF_TOKEN")

login(token=hf_token)

In [5]:
model_id = "meta-llama/Llama-3.2-1B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token or tokenizer.bos_token or tokenizer.unk_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={"": 0},
)

# 4. QLoRA config
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [7]:
data_path = "/kaggle/input/data-llama-finetune/bbc_data_llama_finetune.json"

full_dataset = load_dataset("json", data_files=data_path, split="train")
dataset = full_dataset.train_test_split(test_size=0.1)

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
def tokenize(example, max_length=1024):
    # Tạo chuỗi input theo định dạng:
    # "### Prompt:\n<prompt>\n\n### Summary:\n" + summary
    prompt = f"### Prompt:\n{example['prompt']}\n\n### Summary:\n"
    summary = example["summary"]
    
    # Mã hóa riêng prompt và summary (không thêm special tokens)
    prompt_ids = tokenizer.encode(prompt, add_special_tokens=False)
    summary_ids = tokenizer.encode(summary, add_special_tokens=False)
    
    # Kiểm tra tổng số token
    total_length = len(prompt_ids) + len(summary_ids)
    if total_length > max_length:
        overflow = total_length - max_length
        # Ưu tiên giữ lại phần summary; cắt bớt prompt
        if overflow < len(prompt_ids):
            prompt_ids = prompt_ids[:-overflow]
        else:
            prompt_ids = []  # Nếu quá tràn, bỏ hết prompt
    
    # Nối prompt và summary
    input_ids = prompt_ids + summary_ids
    attention_mask = [1] * len(input_ids)
    
    # Tạo labels: phần prompt được mask bằng -100, phần summary giữ nguyên token IDs
    labels = [-100] * len(prompt_ids) + summary_ids
    
    # Padding tất cả các trường về độ dài max_length
    padding_length = max_length - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + [tokenizer.pad_token_id] * padding_length
        attention_mask = attention_mask + [0] * padding_length
        labels = labels + [-100] * padding_length
    else:
        # Nếu quá dài, cắt bớt (nên không xảy ra nhờ truncation ở trên)
        input_ids = input_ids[:max_length]
        attention_mask = attention_mask[:max_length]
        labels = labels[:max_length]
    
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }
tokenized_dataset = dataset.map(tokenize, batched=False, fn_kwargs={"max_length": 1024})
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

collator = default_data_collator

train_loader = DataLoader(
    tokenized_dataset["train"],
    batch_size=4,
    shuffle=True,
    collate_fn=collator
)

val_loader = DataLoader(
    tokenized_dataset["test"],
    batch_size=4,
    shuffle=False,
    collate_fn=collator
)


Map:   0%|          | 0/1914 [00:00<?, ? examples/s]

Map:   0%|          | 0/213 [00:00<?, ? examples/s]

In [9]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): L

In [10]:
n_epochs = 3

for epoch in range(n_epochs):
    print(f"Epoch {epoch+1}/{n_epochs}")
    model.train()
    pbar = tqdm(train_loader)
    total_loss = 0

    for batch in pbar:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        pbar.set_description(f"Loss: {loss.item():.4f}")

    avg_loss = total_loss / len(train_loader)
    print(f"✅ Epoch {epoch+1} completed — Avg loss: {avg_loss:.4f}")

Epoch 1/3


  0%|          | 0/479 [00:00<?, ?it/s]`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
Loss: 0.1117: 100%|██████████| 479/479 [1:15:30<00:00,  9.46s/it]


✅ Epoch 1 completed — Avg loss: 0.3874
Epoch 2/3


Loss: 0.6455: 100%|██████████| 479/479 [1:15:40<00:00,  9.48s/it]


✅ Epoch 2 completed — Avg loss: 0.3336
Epoch 3/3


Loss: 0.5744: 100%|██████████| 479/479 [1:15:32<00:00,  9.46s/it]

✅ Epoch 3 completed — Avg loss: 0.3229


In [11]:
output_dir = "/kaggle/working/llama3-qlora-finetuned"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print("✅ Finetuned model saved at:", output_dir)

✅ Finetuned model saved at: /kaggle/working/llama3-qlora-finetuned


In [12]:
def evaluate_after_training(model, val_loader, tokenizer, device, num_samples=5):
    import evaluate
    from torch.nn.utils.rnn import pad_sequence

    model.eval()
    predictions = []
    references = []
    samples_to_print = []

    # Load ROUGE
    rouge = evaluate.load("rouge")

    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"

    def left_pad(inputs, pad_token_id):
        """Chuyển batch input thành left-padded"""
        max_len = max(len(seq) for seq in inputs)
        return torch.stack([
            torch.cat([torch.full((max_len - len(seq),), pad_token_id, dtype=torch.long), seq])
            for seq in inputs
        ])

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Evaluating"):
            input_ids = batch["input_ids"]
            attention_mask = batch["attention_mask"]
            labels = batch["labels"]

            # Chuyển sang left padding thủ công
            input_ids = left_pad([x[x != tokenizer.pad_token_id] for x in input_ids], tokenizer.pad_token_id).to(device)
            attention_mask = (input_ids != tokenizer.pad_token_id).long()

            # Generate
            generated_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=128,
                num_beams=4,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id
            )

            # Decode
            generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

            label_texts = []
            for label in labels:
                label_ids = [token_id for token_id in label.tolist() if token_id != -100]
                label_texts.append(tokenizer.decode(label_ids, skip_special_tokens=True))

            predictions.extend(generated_texts)
            references.extend(label_texts)

            if len(samples_to_print) < num_samples:
                src_texts = tokenizer.batch_decode(input_ids, skip_special_tokens=True)
                for src, ref, pred in zip(src_texts, label_texts, generated_texts):
                    samples_to_print.append((src, ref, pred))
                    if len(samples_to_print) >= num_samples:
                        break

    # Tính ROUGE
    results = rouge.compute(predictions=predictions, references=references, use_stemmer=True)

    print("\n📊 ROUGE Scores:")
    for key in results:
        print(f"{key}: {results[key]:.4f}")

    print("\n📝 Sample Results:")
    for i, (src, ref, pred) in enumerate(samples_to_print):
        print(f"\n--- Sample {i+1} ---")
        print(f"[Prompt]    {src}")
        print(f"[Reference] {ref}")
        print(f"[Generated] {pred}")


In [13]:
evaluate_after_training(model, val_loader, tokenizer, device)

Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Evaluating: 100%|██████████| 54/54 [28:22<00:00, 31.53s/it]



📊 ROUGE Scores:
rouge1: 0.4274
rouge2: 0.4257
rougeL: 0.4274
rougeLsum: 0.4271

📝 Sample Results:

--- Sample 1 ---
[Prompt]    ### Prompt:
system
You are a helpful summarization assistant. Provide a concise and accurate summary of the article.
user
Summarize the following article:
Norway upholds 'Napster' ruling

A Norwegian student who ran a website which linked to downloadable MP3 files has been ordered to pay compensation by the country's Supreme Court.

Frank Allan Bruvik was ordered to pay 100,000 kroner (£8,000) to the music industry in Norway. He was a student when he set up his napster.no site, which allowed users to submit and receive links to MP3 files. Bruvik had earlier been cleared on appeal after a lower court had found for the music industry. Music industry bosses in Norway said the ruling would help build confidence in the internet as a distribution medium.

Frank Allan Bruvik set up the napster.no website as part of a school project in 2001 while studying computer en

In [ ]:
# Cấu hình quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
# Load config PEFT
adapter_dir = "/kaggle/working/llama3-qlora-finetuned"
peft_config = PeftConfig.from_pretrained(adapter_dir)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

# Load mô hình gốc và gắn adapter
base_model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    quantization_config=bnb_config,
    device_map={"": 0},
)
model = PeftModel.from_pretrained(base_model, adapter_dir)

In [14]:
data_path = "/kaggle/input/data-llama-finetune/vi_data_llama_finetune.json"

full_dataset = load_dataset("json", data_files=data_path, split="train")
dataset = full_dataset.train_test_split(test_size=0.1)

Generating train split: 0 examples [00:00, ? examples/s]

In [15]:
def tokenize(example, max_length=1024):
    # Tạo chuỗi input theo định dạng:
    # "### Prompt:\n<prompt>\n\n### Summary:\n" + summary
    prompt = f"### Prompt:\n{example['prompt']}\n\n### Summary:\n"
    summary = example["summary"]
    
    # Mã hóa riêng prompt và summary (không thêm special tokens)
    prompt_ids = tokenizer.encode(prompt, add_special_tokens=False)
    summary_ids = tokenizer.encode(summary, add_special_tokens=False)
    
    # Kiểm tra tổng số token
    total_length = len(prompt_ids) + len(summary_ids)
    if total_length > max_length:
        overflow = total_length - max_length
        # Ưu tiên giữ lại phần summary; cắt bớt prompt
        if overflow < len(prompt_ids):
            prompt_ids = prompt_ids[:-overflow]
        else:
            prompt_ids = []  # Nếu quá tràn, bỏ hết prompt
    
    # Nối prompt và summary
    input_ids = prompt_ids + summary_ids
    attention_mask = [1] * len(input_ids)
    
    # Tạo labels: phần prompt được mask bằng -100, phần summary giữ nguyên token IDs
    labels = [-100] * len(prompt_ids) + summary_ids
    
    # Padding tất cả các trường về độ dài max_length
    padding_length = max_length - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + [tokenizer.pad_token_id] * padding_length
        attention_mask = attention_mask + [0] * padding_length
        labels = labels + [-100] * padding_length
    else:
        # Nếu quá dài, cắt bớt (nên không xảy ra nhờ truncation ở trên)
        input_ids = input_ids[:max_length]
        attention_mask = attention_mask[:max_length]
        labels = labels[:max_length]
    
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }
tokenized_dataset = dataset.map(tokenize, batched=False, fn_kwargs={"max_length": 1024})
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

collator = default_data_collator

train_loader = DataLoader(
    tokenized_dataset["train"],
    batch_size=4,
    shuffle=True,
    collate_fn=collator
)

val_loader = DataLoader(
    tokenized_dataset["test"],
    batch_size=4,
    shuffle=False,
    collate_fn=collator
)


Map:   0%|          | 0/144 [00:00<?, ? examples/s]

Map:   0%|          | 0/17 [00:00<?, ? examples/s]

In [16]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): L

In [17]:
n_epochs = 3

for epoch in range(n_epochs):
    print(f"Epoch {epoch+1}/{n_epochs}")
    model.train()
    pbar = tqdm(train_loader)
    total_loss = 0

    for batch in pbar:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        pbar.set_description(f"Loss: {loss.item():.4f}")

    avg_loss = total_loss / len(train_loader)
    print(f"✅ Epoch {epoch+1} completed — Avg loss: {avg_loss:.4f}")

Epoch 1/3


Loss: 0.4241: 100%|██████████| 36/36 [05:46<00:00,  9.64s/it]


✅ Epoch 1 completed — Avg loss: 0.8198
Epoch 2/3


Loss: 0.6029: 100%|██████████| 36/36 [05:45<00:00,  9.60s/it]


✅ Epoch 2 completed — Avg loss: 0.7560
Epoch 3/3


Loss: 0.5810: 100%|██████████| 36/36 [05:45<00:00,  9.59s/it]

✅ Epoch 3 completed — Avg loss: 0.7384


In [18]:
output_dir = "/kaggle/working/llama3-qlora-finetuned-all"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print("✅ Finetuned model saved at:", output_dir)

✅ Finetuned model saved at: /kaggle/working/llama3-qlora-finetuned-all


In [19]:
def evaluate_after_training(model, val_loader, tokenizer, device, num_samples=5):
    model.eval()
    predictions = []
    references = []
    samples_to_print = []

    rouge = evaluate.load("rouge")

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(device)
            labels = batch["labels"]

            # Sinh văn bản
            generated_ids = model.generate(
                input_ids=input_ids,
                max_new_tokens=128,
                num_beams=4,
                do_sample=False
            )

            # Giải mã
            generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
            label_texts = []
            for label in labels:
                label_ids = [token_id for token_id in label.tolist() if token_id != -100]
                label_texts.append(tokenizer.decode(label_ids, skip_special_tokens=True))

            predictions.extend(generated_texts)
            references.extend(label_texts)

            # In ra ví dụ
            if len(samples_to_print) < num_samples:
                for src, ref, pred in zip(tokenizer.batch_decode(input_ids, skip_special_tokens=True), label_texts, generated_texts):
                    samples_to_print.append((src, ref, pred))
                    if len(samples_to_print) >= num_samples:
                        break

    # Tính ROUGE
    results = rouge.compute(predictions=predictions, references=references, use_stemmer=True)

    print("\n📊 ROUGE Scores:")
    for key in results:
        print(f"{key}: {results[key]:.4f}")

    print("\n📝 Sample Results:")
    for i, (src, ref, pred) in enumerate(samples_to_print):
        print(f"\n--- Sample {i+1} ---")
        print(f"[Prompt]    {src}")
        print(f"[Reference] {ref}")
        print(f"[Generated] {pred}")

In [20]:
evaluate_after_training(model, val_loader, tokenizer, device)

Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Evaluating:  20%|██        | 1/5 [00:34<02:17, 34.37s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end 


📊 ROUGE Scores:
rouge1: 0.2803
rouge2: 0.2793
rougeL: 0.2804
rougeLsum: 0.2803

📝 Sample Results:

--- Sample 1 ---
[Prompt]    ### Prompt:
system
Bạn là một trợ lý tóm tắt văn bản hữu ích. Hãy cung cấp bản tóm tắt ngắn gọn và chính xác cho bài viết sau.
user
Hãy tóm tắt bài viết sau:
﻿Hãi hùng nhìn "chạch lươn" ở điểm nóng tai nạn thảm khốc
Hình ảnh PV Dân trí ghi lại dưới đây lý giải vì sao quốc lộ 1A đoạn qua địa bàn huyện Kỳ Anh (Hà Tĩnh) bị xem là cung đường xảy ra nhiều vụ tai nạn thảm khốc nhất trên cả nước.
Nhiều tháng qua, giới tài xế và người dân thường xuyên qua lại tuyến quốc lộ 1A đoạn qua huyện Kỳ Anh - một trong những địa bàn xảy ra nhiều vụ tai nạn thảm khốc -luôn nơm nớp lo sợ tai nạn bất ngờ ập đến. Nguyên nhân do đoạn đường này đã xuống cấp trầm trọng.
Có mặt trên tuyến đường này chúng tôi không khỏi giật mình trước thảm trạng, nhiều đoạn mặt đường bị lún sâu tạo thành vô số "con chạch", nằm chình ình trở thành những cái bẫy hết sức nguy hiểm. Đến đoạn